In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
#import pandas inorder to create dataframes

In [2]:
r=requests.get("https://engineering.careers360.com/colleges/list-of-engineering-colleges-in-india?page=65&sort_by=4")

In [3]:
# Check if the request was successful (status code 200)
if r.status_code == 200:
    soup = BeautifulSoup(r.content, 'html.parser')
print(soup)


<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0" name="viewport"/>
<meta content="LGYIP8lGL_sS1s8J67ez2TByaFYHe22sDZW7IiqhXB4" name="google-site-verification"/>
<meta content="658BD2BEC58D06790B07B5BC052DEAC6" name="msvalidate.01"/>
<title>Page 65 - Engineering Colleges in India 2025 – Courses, Fees, Admission, Rank</title><meta content="Page 65 - Engineering Colleges in India 2025 – Courses, Fees, Admission, Rank" name="name"/><meta content="Page 65 - Check out list of top Engineering colleges in India with courses, fees, cut-off, admission, placement, reviews, ranking, latest news, and more on careers360.com." name="description"/><link href="https://engineering.careers360.com/colleges/list-of-engineering-colleges-in-india?page=65" rel="canonical"/><meta content="colleges list" property="og:type"/><meta content="colleges/list-of-engineering-c

In [4]:
# Extract all <h3> elements with the specified class
college_name=[]
names = soup.find_all('h3', class_='college_name')

# Process and extract college names, splitting by ',' and storing the results in a list
college_names = [names[name].text.strip().split(',')[0].strip() for name in range(0,len(names),2)]
college_name.extend(college_names)
# Print the final list of college names
print(college_name)
print(len(college_names))

['Government Polytechnic', 'Government Polytechnic', 'Government Polytechnic', 'Government Polytechnic', 'Government Polytechnic', 'Government Polytechnic', 'Government Polytechnic', 'Government Polytechnic', 'Government Polytechnic', 'Government Polytechnic', 'Government Polytechnic', 'Government Polytechnic', 'Government Polytechnic', 'Government Polytechnic', 'Government Polytechnic', 'Government Polytechnic', 'Government Polytechnic', 'Government Polytechnic', 'Government Polytechnic', 'Government Polytechnic', 'Government Polytechnic', 'Government Polytechnic', 'Government Ramchandra Khaitan Polytechnic College', 'Government Residence Women Polytechnic', "Government Residential Women's Polytechnic College", "Government Residential Women's Polytechnic College", "Government Residential Women's Polytechnic", 'Government Rural Polytechnic', 'Government Science College', 'MORAR Gwalior - Government Shyamlal Pandviya PG College', 'Government Sri Krishnarajendra Silver Jubilee Technologi

In [5]:
# Lists to store the details
city_list = []
state_list = []
type_list = []

# Extract city, state, and type from the HTML content
cities = soup.find_all('div', class_='content_block d-none d-md-block d-md-flex flex-row justify-content-between')
for i, div in enumerate(cities):
    spans = div.find_all('span')
    span_texts = [span.text.strip() for span in spans]

    # Extract city and state
    location = span_texts[0].split(',') if len(span_texts) > 0 else ["Unknown", "Unknown"]
    city = location[0].strip()
    state = location[1].strip() if len(location) > 1 else "Unknown"

    # Extract type (Private/Public)
    second_object = span_texts[1] if len(span_texts) > 1 else "Unknown"
    college_type = "Private" if 'private' in second_object.lower() else \
                   "Public" if 'public' in second_object.lower() or 'govt' in second_object.lower() else \
                   "Unknown"

    # Append details to the respective lists
    city_list.append(city)
    state_list.append(state)
    type_list.append(college_type)

# Print the lists to verify the results
# print(len(city_list))
# print(len(state_list))
# print(len(type_list))


In [6]:
# List to store the fees details
fees_list = []

# Find all div elements with the class 'snippet_block'
fees_blocks = soup.find_all('div', class_='snippet_block')

# Iterate over each 'div'
for div in fees_blocks:
    # Find all 'li' elements within the current 'div'
    list_items = div.find_all('li')
    
    # Initialize a variable to store fees
    fees_text = None
    
    # Check each 'li' for the word 'Fees'
    for li in list_items:
        if 'fees' in li.text.lower():  # Case-insensitive check for 'Fees'
            fees_text = li.text.strip().replace("Fees :  ₹", "").strip()  # Extract and clean the text
            break  # Stop searching once 'Fees' is found
    
    # Append fees to the list or 'None' if not found
    fees_list.append(fees_text if fees_text else "None")

# Print the fees list to verify
#print("Fees List:", fees_list)
#print("Number of Fees Entries:", len(fees_list))


In [7]:
# Find all 'div' elements with class 'important_links'
links_blocks = soup.find_all('div', class_='important_links')

# Initialize lists to store the links
courses_links = []
facilities_links = []

# Iterate over each 'div' element
for div in links_blocks:
    # Find all 'li' elements within the current 'div'
    list_items = div.find_all('li')
    
    # Initialize variables to store the links for the current block
    course_link = None
    facility_link = None
    
    # Check each 'li' element for the relevant text and extract links
    for li in list_items:
        if 'courses' in li.text.lower():  # Case-insensitive check for 'Courses'
            a_tag = li.find('a', href=True)  # Find the 'a' tag with href
            course_link = a_tag['href'] if a_tag else None
        elif 'facilities' in li.text.lower():  # Case-insensitive check for 'Facilities'
            a_tag = li.find('a', href=True)  # Find the 'a' tag with href
            facility_link = a_tag['href'] if a_tag else None

    # Append the extracted links or None to the respective lists
    courses_links.append(course_link)
    facilities_links.append(facility_link)
    
# # Print the results
#print("Courses Links:", courses_links)
# print(len(courses_links))
# print("Facilities Links:", facilities_links)

In [8]:
import requests
from bs4 import BeautifulSoup
import re

# List to store the formatted data
no_of_courses = []
course_names = []

# Loop over only even indices (0, 2, 4, etc.)
for i in range(0, len(courses_links), 2):  # Start at index 0 and increment by 2
    url = courses_links[i]
    
    try:
        # Send the GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raises HTTPError for bad responses (4xx or 5xx)
        
        # Parse the page content using BeautifulSoup
        course_soup = BeautifulSoup(response.content, 'html.parser')

        # Extract the college name from the URL
        college_name_part = ''
        if 'colleges' in url:
            college_name_part = url.split("/colleges/")[1].split("/")[0]
        elif 'university' in url:
            college_name_part = url.split("/university/")[1].split("/")[0]
        
        # Replace hyphens with spaces to get the college name
        college_name = college_name_part.replace("-", " ")

        # Extract course count
        course_count_div = course_soup.find('div', class_='course_count')
        if course_count_div:
            # Using regex to extract the number
            match = re.search(r'\d+', course_count_div.text.strip())
            course_count = int(match.group()) if match else 0
        else:
            course_count = 0  # Default to 0 if course count is not found
        no_of_courses.append(course_count)

        # Extract course names
        course_name_divs = course_soup.find_all('div', class_='card_blk')
        if course_name_divs:
            courses = [name.text.strip().split("Offered by")[0].strip() for name in course_name_divs]
        else:
            courses = []  # Set to an empty list if no courses are found
        course_names.append(courses)

    except requests.exceptions.RequestException as e:
        # Handle all request-related exceptions (e.g., network issues, invalid URLs)
        print(f"Request failed for URL: {url}. Error: {e}")
        no_of_courses.append(None)  # Append None if request fails
        course_names.append(None)  # Append None if request fails

    except Exception as e:
        # Handle any other exceptions during the parsing or processing steps
        print(f"An error occurred while processing {url}. Error: {e}")
        no_of_courses.append(None)  # Append None for any other error
        course_names.append(None)  # Append None for any other error

# Now `no_of_courses` and `course_names` contain the desired output, with None appended for failures
#print(no_of_courses, course_names)


Request failed for URL: https://www.careers360.com/colleges/government-tool-room-and-training-centre-dumka/courses. Error: 404 Client Error: Not Found for url: https://www.careers360.com/colleges/government-tool-room-and-training-centre-dumka/courses


In [9]:
import requests
from bs4 import BeautifulSoup

# List to store the formatted data
college_facilities = []

# Loop over only even indices (0, 2, 4, etc.)
for i in range(0, len(facilities_links), 2):  # Start at index 0 and increment by 2
    url = facilities_links[i]
    
    try:
        # Check if the URL is valid
        if not url or not url.startswith(("http://", "https://")):
            print(f"Skipping invalid URL: {url}")
            college_facilities.append(None)  # Append None for skipped URL
            continue  # Skip to the next iteration if URL is invalid
        
        # Send the GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raises HTTPError for bad responses (4xx or 5xx)
        
        # Parse the page content using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract the college name from the URL
        college_name_part = ''
        if 'colleges' in url:
            college_name_part = url.split("/colleges/")[1].split("/")[0]
        elif 'university' in url:
            college_name_part = url.split("/university/")[1].split("/")[0]
        
        # Replace hyphens with spaces to get the college name
        college_name = college_name_part.replace("-", " ")

        # Find all li elements under the div with class 'facilities_list'
        facility_list_div = soup.find('div', class_='facilities_list')
        if facility_list_div:
            facility_names = facility_list_div.find_all('li')
            # Extract facility names
            facility_texts = [
                li.find('span', class_='facilities_name').text.strip() 
                for li in facility_names if li.find('span', class_='facilities_name')
            ]
        else:
            facility_texts = []  # Default to an empty list if no facilities are found
        
        # Append the extracted facility names to the college_facilities list
        college_facilities.append(facility_texts)

    except requests.exceptions.RequestException as e:
        # Handle all request-related exceptions (e.g., network issues, invalid URLs)
        print(f"Request failed for URL: {url}. Error: {e}")
        college_facilities.append(None)  # Append None when request fails

    except Exception as e:
        # Handle other potential exceptions during the parsing or processing steps
        print(f"An error occurred while processing {url}. Error: {e}")
        college_facilities.append(None)  # Append None for any other error

# Print the result
#print(college_facilities)


Skipping invalid URL: None
Skipping invalid URL: None


In [10]:
print(len(college_name))
print(len(city_list))
print(len(state_list))
print(len(type_list))
print(len(fees_list))


47
50
50
50
50


In [11]:
# import pandas as pd
# import re

# course_names = []
# facilities = []
# no_of_courses_list = []  # List to store the number of courses

# # Populate the course_names, facilities, and no_of_courses columns
# for college in college_name:
#     # Check for matching course names and no_of_courses using regex
#     matched_courses = None
#     matched_no_of_courses = None
#     for entry in formatted_college_data:
#         course_college, count_and_courses = entry.split(": ", 1)
#         count_of_courses, offered_course_names = count_and_courses.split(",", 1)
#         if re.search(rf"\b{re.escape(college)}\b", course_college, re.IGNORECASE):
#             matched_courses = offered_course_names
#             matched_no_of_courses = count_of_courses  # Extract no_of_courses
#             break
#     course_names.append(matched_courses if matched_courses else None)
#     no_of_courses_list.append(matched_no_of_courses if matched_no_of_courses else None)
    
#     # Check for matching facilities using regex
#     matched_facilities = None
#     for facility_entry in formatted_college_facilities:
#         facility_college, facility_values = facility_entry.split(": ", 1)
#         if re.search(rf"\b{re.escape(college)}\b", facility_college, re.IGNORECASE):
#             matched_facilities = facility_values
#             break
#     facilities.append(matched_facilities if matched_facilities else None)

# # Create a dictionary for the dataset
# data = {
#     "College Name": college_name,
#     "City": city_list,
#     "State": state_list,
#     "Type": type_list,
#     "Fees": fees_list,
#     "No of Courses": no_of_courses_list,  # Use the updated list
#     "Course Names": course_names,
#     "Facilities": college_facilities
# }

# # Convert to DataFrame
# df = pd.DataFrame(data)

# # Display the DataFrame
# print(df)

# # Save to CSV (optional)
# df.to_csv("colleges_dataset.csv", index=False)

# print("Dataset saved to 'colleges_dataset.csv'")


In [12]:
# Create a dictionary for the dataset
data = {
    "College Name": college_names,
    "City": city_list,
    "State": state_list,
    "Type": type_list,
    "Fees": fees_list,
    "No of Courses": no_of_courses,  # Use the updated list
    "Course Names": course_names,
    "Facilities": college_facilities
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

# Save to CSV (optional)
df.to_csv("colleges_dataset30.csv", index=False)
print("Dataset saved to 'colleges_dataset.csv'")

                                         College Name                 City  \
0                              Government Polytechnic                Unnao   
1                              Government Polytechnic            Anantapur   
2                              Government Polytechnic           Uttarkashi   
3                              Government Polytechnic              Uttawar   
4                              Government Polytechnic               Gadwal   
5                              Government Polytechnic              Mehsana   
6                              Government Polytechnic             Vaishali   
7                              Government Polytechnic               Valsad   
8                              Government Polytechnic               Kadapa   
9                              Government Polytechnic           Vijayawada   
10                             Government Polytechnic           Rangareddy   
11                             Government Polytechnic           

In [13]:
#print(facilities_links)

In [14]:
#facilities_links[6]

In [15]:
# # Finding None values in facilities_links
# none_indices = [i for i, link in enumerate(facilities_links) if link is None]

# # Print the indices where None is found
# print("Indices with None:", none_indices)

# # Print the None values (if any)
# none_values = [link for link in facilities_links if link is None]
# print("None values in the list:", none_values)


In [16]:
#college_names[42]

In [17]:
len(course_names)

50

In [18]:
#len(college_facilities)

In [235]:
import pandas as pd
import random

# Load the CSV file into a DataFrame
df = pd.read_excel("all_Combined.xlsx")

# Function to generate a random fee value
def generate_random_fee():
    # Randomly decide if the fee should be in 'k' or 'lakhs'
    if random.random() < 0.6:  # 80% chance for 'k'
        fee = random.randint(10, 99)  # Random value between 10k and 99k
        return f"{fee}k"
    else:  # 20% chance for 'lakhs'
        fee = round(random.uniform(1.0, 15.0), 1)  # Random value between 1.0 and 10.0 lakhs
        return f"{fee}lakhs"

# Replace None or NaN in the 'Fees' column with a random fee value
df['Fees'] = df['Fees'].apply(lambda x: generate_random_fee() if pd.isna(x) or x in [None, "None"] else x)

# Save the updated DataFrame back to the CSV file
df.to_csv("colleges_dataset_fees_updated.csv", index=False)

print("Updated dataset saved to 'colleges_dataset_fees_updated.csv'.")


Updated dataset saved to 'colleges_dataset_fees_updated.csv'.


In [236]:
import pandas as pd
import random

# Load the Excel file into a DataFrame
df = pd.read_excel("all_Combined.xlsx")

# Function to generate a random fee value
def generate_random_fee(college_type):
    if college_type and "Public" in college_type:
        # Generate a fee value in 'k' for public colleges
        fee = random.randint(10, 99)  # Random value between 10k and 99k
        return f"{fee}k"
    else:
        # Generate a fee value in 'lakhs' for other colleges
        fee = round(random.uniform(1.0, 15.0), 1)  # Random value between 1.0 and 15.0 lakhs
        return f"{fee}lakhs"

# Replace None or NaN in the 'Fees' column based on the value of the 'Type' column
df['Fees'] = df.apply(
    lambda row: generate_random_fee(row['Type']) if pd.isna(row['Fees']) or row['Fees'] in [None, "None"] else row['Fees'],
    axis=1
)

# Save the updated DataFrame back to a CSV file
df.to_csv("colleges_dataset_fees_updated.csv", index=False)

print("Updated dataset saved to 'colleges_dataset_fees_updated.csv'.")


Updated dataset saved to 'colleges_dataset_fees_updated.csv'.


In [237]:
import pandas as pd
import random

# Load the Excel file into a DataFrame
df = pd.read_excel("all_Combined.xlsx")

# Function to generate a random fee value
def generate_random_fee(college_type):
    if college_type and "Public" in college_type:
        # Generate a fee value strictly between 10k and 2lakhs for public colleges
        if random.random() < 0.7:  # 80% chance for 'k'
            fee = random.randint(10, 99)  # Random value between 10k and 99k
            return f"{fee}k"
        else:  # 20% chance for 'lakhs' within the 1-2 lakh range
            fee = round(random.uniform(1.0, 2.0), 1)  # Random value between 1.0 and 2.0 lakhs
            return f"{fee}lakhs"
    else:
        # Generate a fee value strictly between 10k and 15lakhs for private colleges
        if random.random() < 0.6:  # 40% chance for 'k'
            fee = random.randint(10, 99)  # Random value between 10k and 99k
            return f"{fee}k"
        else:  # 60% chance for 'lakhs' within the 1-15 lakh range
            fee = round(random.uniform(1.0, 15.0), 1)  # Random value between 1.0 and 15.0 lakhs
            return f"{fee}lakhs"

# Replace None or NaN in the 'Fees' column based on the value of the 'Type' column
df['Fees'] = df.apply(
    lambda row: generate_random_fee(row['Type']) if pd.isna(row['Fees']) or row['Fees'] in [None, "None"] else row['Fees'],
    axis=1
)

# Save the updated DataFrame back to a CSV file
df.to_csv("colleges_dataset_fees_updated.csv", index=False)

print("Updated dataset saved to 'colleges_dataset_fees_updated.csv'.")


Updated dataset saved to 'colleges_dataset_fees_updated.csv'.


In [14]:
"""
Fetches college data from multiple pages and returns a list of BeautifulSoup objects for each page.

Args:
max_pages (int): The maximum number of pages to fetch.

Returns:
list: A list of BeautifulSoup objects for each page.
"""
base_url = "https://engineering.careers360.com/colleges/list-of-engineering-colleges-in-india?page={}&sort_by=4"
college_data = []
max_pages = 3
for page in range(1, max_pages + 1):  # Loop through pages starting from 1 to max_pages
    url = base_url.format(page)
    response = requests.get(url)

    if response.status_code == 200:  # Check if the request was successful
        soup = BeautifulSoup(response.content, 'html.parser')
        college_data.append(soup)
        print(f"Successfully fetched data for page {page}")
    else:
        print(f"Failed to fetch data for page {page}, status code: {response.status_code}")
        break
print(len(college_data))

Successfully fetched data for page 1
Successfully fetched data for page 2
Successfully fetched data for page 3
3
